In [ ]:
from datasets.cwru import CWRU
from utils import (setlogger, fast_adapt)

import torch
import logging
import random
import numpy as np
import learn2learn as l2l
import matplotlib.pyplot as plt

from torch import nn
from learn2learn.data.transforms import (
    FusedNWaysKShots,
    LoadData,
    RemapLabels,
    ConsecutiveLabels,
)

In [ ]:
# N-way, K-shot few-Shot learning parameters
ways=10             # Number of classes in a task
shots=5             # Number of training examples per class (same for testing)
# Meta-learning parameters
meta_lr=0.001       # Outer loop learning rate
fast_lr=0.1         # Inner loop learning rate
adapt_steps=5       # Number of inner loop update steps
meta_batch_size=32  # Number of tasks sampled per batch
iterations=5        # Number of outer loop iterations
# Cuda and random seed settings
cuda=True
seed=42
# Dataset parameters (different domain means different working condition)
train_domain1=0      # For CWRU dataset: 0: 1797, 1: 1772, 2: 1750, 3: 1730
train_domain2=1

valid_domain=2     
test_domain=3
# Path to the data directory
data_dir_path='./data'

In [ ]:
# Set the Random Seed
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# Set training device, using GPU if available
if cuda and torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    device_count = torch.cuda.device_count()
    device = torch.device('cuda')
    logging.info('Training MAML with {} GPU(s).'.format(device_count))
else:
    device = torch.device('cpu')
    logging.info('Training MAML with CPU.')

In [ ]:
# Create Datasets
train_dataset1 = CWRU(train_domain1,
                        data_dir_path)
train_dataset2 = CWRU(train_domain2,
                        data_dir_path)

valid_dataset = CWRU(valid_domain,
                        data_dir_path)

test_dataset = CWRU(test_domain,
                    data_dir_path)

# Create Meta-Datasets
train_dataset1 = l2l.data.MetaDataset(train_dataset1)
train_dataset2 = l2l.data.MetaDataset(train_dataset2)
valid_dataset = l2l.data.MetaDataset(valid_dataset)
test_dataset = l2l.data.MetaDataset(test_dataset)

# Create Meta-Tasks
train_transforms = [
    FusedNWaysKShots(train_dataset1, n=ways, k=2*shots),
    LoadData(train_dataset1),
    RemapLabels(train_dataset1),
    ConsecutiveLabels(train_dataset1),
]
train_tasks1 = l2l.data.Taskset(
    train_dataset1,
    task_transforms=train_transforms,
    num_tasks=400,
)
train_transforms = [
    FusedNWaysKShots(train_dataset2, n=ways, k=2*shots),
    LoadData(train_dataset2),
    RemapLabels(train_dataset2),
    ConsecutiveLabels(train_dataset2),
]
train_tasks2 = l2l.data.Taskset(
    train_dataset2,
    task_transforms=train_transforms,
    num_tasks=400,
)

valid_transforms = [
    FusedNWaysKShots(valid_dataset, n=ways, k=2*shots),
    LoadData(valid_dataset),
    ConsecutiveLabels(valid_dataset),
    RemapLabels(valid_dataset),
]
valid_tasks = l2l.data.Taskset(
    valid_dataset,
    task_transforms=valid_transforms,
    num_tasks=100,
)

test_transforms = [
    FusedNWaysKShots(test_dataset, n=ways, k=2*shots),
    LoadData(test_dataset),
    RemapLabels(test_dataset),
    ConsecutiveLabels(test_dataset),
]
test_tasks = l2l.data.Taskset(
    test_dataset,
    task_transforms=test_transforms,
    num_tasks=100,
)

In [ ]:
# Create Model
model = l2l.vision.models.CNN4(output_size=10)
model.to(device)
maml = l2l.algorithms.MAML(model, lr=fast_lr, first_order=False)
opt = torch.optim.Adam(model.parameters(), meta_lr)
loss = nn.CrossEntropyLoss(reduction='mean')

In [ ]:
train_acc = []
valid_acc = []
train_loss = []
valid_loss = []

In [ ]:
for iteration in range(1, iterations+1):
    opt.zero_grad()
    meta_train_error = 0.0
    meta_train_accuracy = 0.0
    meta_valid_error = 0.0
    meta_valid_accuracy = 0.0
    meta_test_error = 0.0
    meta_test_accuracy = 0.0

    for task in range(meta_batch_size):
        # Compute meta-training loss
        learner = maml.clone()
        batch = train_tasks1.sample()
        evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                            learner,
                                                            loss,
                                                            adapt_steps,
                                                            shots,
                                                            ways,
                                                            device)
        evaluation_error.backward()
        meta_train_error += evaluation_error.item()
        meta_train_accuracy += evaluation_accuracy.item()
        
        learner = maml.clone()
        batch = train_tasks2.sample()
        evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                            learner,
                                                            loss,
                                                            adapt_steps,
                                                            shots,
                                                            ways,
                                                            device)
        evaluation_error.backward()
        meta_train_error += evaluation_error.item()
        meta_train_accuracy += evaluation_accuracy.item()
        
        # Compute meta-validation loss
        learner = maml.clone()
        batch = valid_tasks.sample()
        evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                            learner,
                                                            loss,
                                                            adapt_steps,
                                                            shots,
                                                            ways,
                                                            device)
        meta_valid_error += evaluation_error.item()
        meta_valid_accuracy += evaluation_accuracy.item()

        # # Compute meta-testing loss
        # learner = maml.clone()
        # batch = test_tasks.sample()
        # evaluation_error, evaluation_accuracy = fast_adapt(batch,
        #                                                     learner,
        #                                                     loss,
        #                                                     adapt_steps,
        #                                                     shots,
        #                                                     ways,
        #                                                     device)
        # meta_test_error += evaluation_error.item()
        # meta_test_accuracy += evaluation_accuracy.item()

    train_acc.append(meta_train_accuracy / meta_batch_size)
    valid_acc.append(meta_valid_accuracy / meta_batch_size)
    train_loss.append(meta_train_error / meta_batch_size)
    valid_loss.append(meta_valid_error / meta_batch_size)

    # Print some metrics
    # if (iteration % 10 == 0):
    print('\n')
    print('Iteration', iteration)
    print('Meta Train Error', meta_train_error / meta_batch_size)
    print('Meta Train Accuracy', meta_train_accuracy / meta_batch_size)
    print('Meta Valid Error', meta_valid_error / meta_batch_size)
    print('Meta Valid Accuracy', meta_valid_accuracy / meta_batch_size)
    print('Meta Test Error', meta_test_error / meta_batch_size)
    print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)

    # ========================= plot ==========================
    if (iteration % 1 == 0):
        plt.figure(figsize=(12, 4))
        plt.subplot(121)
        plt.plot(train_acc, '-o', label="train acc")
        plt.plot(valid_acc, '-o', label="valid acc")
        plt.xlabel('Trainin iteration')
        plt.ylabel('Accuracy')
        plt.title("Accuracy Curve by Iteration")
        plt.legend()
        plt.subplot(122)
        plt.plot(train_loss, '-o', label="train loss")
        plt.plot(valid_loss, '-o', label="valid loss")
        plt.xlabel('Trainin iteration')
        plt.ylabel('Loss')
        plt.title("Loss Curve by Iteration")
        plt.legend()
        plt.suptitle("CWRU Bearing Fault Diagnosis {}way-{}shot".format(ways, shots))
        # plt.savefig('./results/image_{}.png'.format(iteration))
        plt.show()

    # Average the accumulated gradients and optimize
    for p in model.parameters():
        p.grad.data.mul_(1.0 / meta_batch_size)
    opt.step()